In [41]:
import datetime as dt
import logging as log

%run BreakSectionManager.ipynb
%run Util.ipynb
%run Constant.ipynb
%run Metric.ipynb
%run Operation.ipynb
%run Account.ipynb
%run Policy_Loader.ipynb


class RocketRegression:
    
    def __init__(self, policy_name):
        self.policy_name = policy_name
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.context = self.policy_loader.get_context()
        
        self.level = self.context['level']
        self.short = self.context['short']
        self.mid = self.context['mid']
        
        
    def start_regression_on_one_stock(self,symbol):
        sectionManager = BreakSectionManager(symbol,self.context)
        section_df = sectionManager.get_all_break_sections()
        
        metric = Metric(symbol,self.context)
        
        stock_start_date = metric.get_stock_start_date()
        stock_end_date = metric.get_stock_end_date()
        log.info('stock_start_date ' + stock_start_date+' stock_end_date:'+stock_end_date)
        
        account = Account(stock_start_date,stock_end_date,self.context,self.policy_name)
        operation = Operation(account,symbol,self.context,self.policy_name)
        
        for key,row in section_df.iterrows():
            start_date = row['d_s_date']
            end_date = row['d_e_date']
            
            start_datetime = to_datetime(start_date)
            end_datetime = to_datetime(end_date)

            buy_price = metric.get_cur_price(start_date)
            
            while(start_datetime<end_datetime):
                cur_date = start_datetime.strftime(YMD_format)
                    
                if(account.can_open_new_stock()==True):
                    if(operation.is_buy_point(cur_date)==True):
                        operation.open_opsition(cur_date)
                        
                if(account.has_shares()==True):
                    if(operation.is_sell_point(cur_date)==True):
                        operation.sell_stock(cur_date)
                        
                start_datetime += dt.timedelta(days = 1)
                        
        operations = account.get_operations()
        
        return operations
    

In [42]:
regression = RocketRegression('rocket_break_baseline.json')

operations = regression.start_regression_on_one_stock('SZ#000001')
operations

2000-03-09 buy SZ#000001 price:3.55 100 position:{'SZ#000001': 100} unknow_market space:0.36
2000-05-25 buy SZ#000001 price:3.28 100 position:{'SZ#000001': 200} unknow_market space:0.66


KeyError: 'SZ#000001'